<a href="https://colab.research.google.com/github/andrepinotti/ICDS6-2024/blob/main/chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

import torch
tokenizer = AutoTokenizer.from_pretrained("microsoft/DialoGPT-medium")
model = AutoModelForCausalLM.from_pretrained("microsoft/DialoGPT-medium")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/642 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/863M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Interação com a IA em 5 linhas, teremos uma conversa básica com encerramento

In [ ]:
for step in range(5):
  new_user_input_ids = tokenizer.encode(input(">> User:") + tokenizer.eos_token, return_tensors='pt')

  bot_input_ids = torch.cat([chat_history_ids, new_user_input_ids], dim=-1) if step > 0 else new_user_input_ids

  chat_history_ids = model.generate(bot_input_ids, max_length=1000, pad_token_id=tokenizer.eos_token_id)

  print("DialoGPT: {}".format(tokenizer.decode(chat_history_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)))


>> User:Hello!
DialoGPT: Hello! :D
>> User:How are you doing?
DialoGPT: I'm doing well! How are you?
>> User:I'm fine
DialoGPT: That's good!
>> User:Have a good day
DialoGPT: You too! :D
>> User:bye
DialoGPT: Bye! :D


Agora criaremos o Chat-Boot:

In [ ]:
import pandas as pd

#Criar um dicionário para o dataframe
dados_pedidos = {
"numero_pedido": ["12345", "67890", "11121", "22232"],
"status": ["Shipped","Processing","Delivered","Cancelled"]
}

#Criar o dataframe com o dicionário
df_status_pedidos = pd.DataFrame(dados_pedidos)

#Verificar se o dataframe foi criado corretamente
print(df_status_pedidos)

  numero_pedido      status
0         12345     Shipped
1         67890  Processing
2         11121   Delivered
3         22232   Cancelled


Desenvolveremos um código que permita consultar o status de um pedido com base no número informado
pelo cliente. Também precisamos lidar com a possibilidade de o cliente digitar um número de pedido
inexistente, e retornar uma mensagem apropriada, como "O número que você inseriu não existe. Por
favor, verifique e tente novamente."


In [ ]:
def verificar_status_pedido(numero_pedido):
  try:
    status = df_status_pedidos[df_status_pedidos["numero_pedido"] == numero_pedido]["status"].iloc[0]
    return f"The status of your order {numero_pedido} is: {status}"
  except:
    return "Order number not found. Please check and try again."



In [ ]:
  print(verificar_status_pedido("12345"))

The status of your order 12345 is: Shipped


Criaremos uma identificação de palavras chaves para consultar pedido. Criando palavras específicas, faremos com que o ChatBot saiba que será necessário consultar o status de um pedido


In [ ]:
palavras_chave_status = ["order", "order status", "status of my order", "check my order", "track my order", "order update"]

Encerramento da conversa

Implementar uma função para encerrar o chat. Se o usuário digitar uma palavra específica, o loop
será interrompido e o chat será encerrado. Para isso, podemos usar uma estrutura if para verificar se
o input_usuario contém uma das palavras de saída, como "exit", "quit", ou "stop".


In [ ]:
while True:
  input_usuario = input("You: ")
  if input_usuario.lower() in ['exit', 'quit', 'stop']:
    print("Bot: Goodbye!")
    break

You: Hi 
You: stop
Bot: Goodbye!


Interagindo com o chatbot:

Consultando pedidos

In [ ]:
ids_historico_chat = None
while True:
  input_usuario = input("You: ")
  if input_usuario.lower() in ['exit', 'quit', 'stop']:
    print("Bot: Goodbye!")
    break

  if any(keyword in input_usuario.lower() for keyword in palavras_chave_status):
    numero_pedido = input('Could you please enter your order number?')
    resposta = verificar_status_pedido(numero_pedido)
    print(f'Bot: {resposta}')
  else:
    novo_usuario_input_ids = tokenizer.encode(input_usuario + tokenizer.eos_token, return_tensors='pt')

    if ids_historico_chat is not None:
      bot_input_ids = torch.cat([ids_historico_chat, novo_usuario_input_ids], dim=-1)
    else:
      bot_input_ids = novo_usuario_input_ids

    ids_historico_chat = model.generate(
      bot_input_ids,
      max_length=1000,
      pad_token_id=tokenizer.eos_token_id
    )
    resposta = tokenizer.decode(ids_historico_chat[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)

    print(f'Bot: {resposta}')

You: hi
Bot: Hey! :D
You: how are you?
Bot: I'm good, how are you?
You: I'm fine, have a good day
Bot: you too
You: order
Could you please enter your order number?12345
Bot: The status of your order 12345 is: Shipped
You: quit
Bot: Goodbye!
